In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate('crp-db-manager/crp-backend-server-b67475fc5e44.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [ ]:
# code to add User details to the database
import pandas as pd

user_df = pd.read_csv('crp-db-manager/user_db.csv')

user_ref = db.collection(u'User')

for index, row in user_df.iterrows():
    doc_ref = user_ref.document(row['Registration Number'])
    
    regno = row['Registration Number']
    uname = row['Username']
    passw = row['Password']
    email = row['Email']
    phnno = row['Phone Number']
    usert = row['User Type']
    
    doc_ref.set({
        u'registrationNumber': regno,
        u'name': uname,
        u'password': passw,
        u'email': email,
        u'phoneNumber': phnno,
        u'userType': usert
    })

In [ ]:
# code to add Course details to the database
import json

course_ref = db.collection(u'Course')

with open('crp-db-manager/course_parse_data.json', 'r') as course_db:
    course_data = json.loads(course_db.read())
    
    for code, info in course_data.items():
        doc_ref = course_ref.document(str(code))
        try:
            doc_ref.set(info)
        except:
            print(info)

In [ ]:
# Code to fetch details

# Available data: User, Course

doc_ref = db.collection(u'Course')

docs = doc_ref.stream()

for doc in docs:
    print(u'{} => {}'.format(doc.id, doc.to_dict()))

In [ ]:
from firebase_admin import auth

user_df = pd.read_csv('crp-db-manager/user_db.csv')

email_passw = user_df[['Email', 'Password']]

users = list()

for i in range(len(email_passw)):
    users.append(
        auth.ImportUserRecord(
            uid='uid' + str(i + 1),
            email=email_passw.iloc[i]['Email'],
            password_hash=bytes(email_passw.iloc[i]['Password'], 'utf-8'),
            password_salt=b'salt' + bytes(i + 1)
        )
    )

In [ ]:

hash_alg = auth.UserImportHash.hmac_sha256(key=b'sooperdooperceekret')
try:
    result = auth.import_users(users, hash_alg=hash_alg)
    for err in result.errors:
        print('Failed to import user:', err.reason)
except exceptions.FirebaseError as error:
    print('Error importing users:', error)


In [ ]:
import pandas as pd

user_df = pd.read_csv('crp-db-manager/user_db.csv')

user_course_list = list()

for i in range(user_df.shape[0]):
    user_course_list.append(user_df.iloc[i]['Registration Number'][8:11])

user_df = user_df.assign(Dept = user_course_list)

In [ ]:
from collections import defaultdict
import json

course_dict = defaultdict(list)

courses = dict()
with open('crp-db-manager/course_parse_data.json', 'r') as course_db:
    courses = json.loads(course_db.read())

for elem in courses.keys():
    course_dict[elem[2:5]].append(elem)

course_dict

In [ ]:
import random

student_details = dict()

for i in range(user_df.shape[0]):
    if user_df.iloc[i]['User Type'] == 'STUDENT':
        if user_df.iloc[i]['Dept'] != 'CIV': 
            course_list = course_dict[user_df.iloc[i]['Dept']] + course_dict['HUM'] + course_dict['MAT']
        else:
            course_list = course_dict['HUM'] + course_dict['MAT']

        total_course = random.randint(0, min(7, len(course_list)))
        random_courses = random.sample(course_list, total_course)

        course_rem_set = list(set(course_list) - set(random_courses))

        course_rem = random.sample(course_rem_set, min(10 - total_course, len(course_rem_set)))
        
        student_details[user_df.iloc[i]['Registration Number']] = {
            'rollNumber': user_df.iloc[i]['Registration Number'],
            'reviews': [],
            'questions': [],
            'coursesCompleted': random_courses,
            'coursesRemaining': course_rem
        }

In [ ]:
student_ref = db.collection(u'Student')

for rollNum, info in student_details.items():
    doc_ref = student_ref.document(str(rollNum))
    try:
        doc_ref.set(info)
    except:
        print(info)

In [ ]:
from collections import defaultdict

subscribe_course_set = set()

for elem in student_details.values():
    subscribe_course_set.update(elem.get('coursesCompleted'))
    subscribe_course_set.update(elem.get('coursesRemaining'))

course_count = defaultdict(int)

for elem in subscribe_course_set:
    course_count[elem[2:5]] += 1

faculty_count = defaultdict(int)
for i in range(user_df.shape[0]):
    
    if user_df.iloc[i]['User Type'] in ['FACULTY', 'MENTOR']:
        faculty_count[user_df.iloc[i]['Registration Number'][8:11]] += 1

In [ ]:
course_count